In [1]:
import torch

device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"
print(f"device: {device}")

device: cpu


In [2]:
# Reading annotations.json

TRAIN_ANNOTATIONS_PATH = "D:/2022_imgdl/data/training/annotations.json"
TRAIN_IMAGE_DIRECTIORY = "D:/2022_imgdl/data/training/images/"

VAL_ANNOTATIONS_PATH = "D:/2022_imgdl/data/validation/annotations.json"
VAL_IMAGE_DIRECTIORY = "D:/2022_imgdl/data/validation/images/"

In [25]:
# Reading the annotation files
import json

with open(TRAIN_ANNOTATIONS_PATH) as f:
  train_annotations_data = json.load(f)

with open(VAL_ANNOTATIONS_PATH) as f:
  val_annotations_data = json.load(f)

In [26]:
# For reading annotations file
import pycocotools
from pycocotools.coco import COCO

train_coco = COCO(TRAIN_ANNOTATIONS_PATH)
val_coco = COCO(VAL_ANNOTATIONS_PATH)

loading annotations into memory...
Done (t=0.28s)
creating index...
index created!


In [ ]:
import matplotlib.pyplot as plt

start_no = 1
end_no = len(train_annotations_data['images'])

for img_no in range(end_no):
    
    annIds = train_coco.getAnnIds(imgIds=train_annotations_data['images'][img_no]['id'])
    anns = train_coco.loadAnns(annIds)

    # load and render the image
    plt.imshow(plt.imread(TRAIN_IMAGE_DIRECTIORY+train_annotations_data['images'][img_no]['file_name']))
    plt.axis('off')
    # Render annotations on top of the image
    train_coco.showAnns(anns)

    mask = train_coco.annToMask(anns[0])
    for i in range(len(anns)):
        mask += train_coco.annToMask(anns[i])
        plt.imshow(mask)
        plt.axis("off")
        plt.savefig(fname='D:/2022_imgdl/data/training/masks/' + str(train_annotations_data['images'][img_no]['id']) + '.png', bbox_inches='tight', pad_inches=0)
    
    # plot 초기화
    plt.clf()

In [ ]:
# change file name
import os

path = "D:/2022_imgdl/data/training/masks"
file_list = os.listdir(path)

for name in file_list:
    newname = name.replace(".png", "")
    newname = newname.zfill(6)

    newname = newname + ".png"
    src = os.path.join(path, name)
    dst = os.path.join(path, newname)
    os.rename(src, dst)

In [30]:
import matplotlib.pyplot as plt

start_no = 1
end_no = len(val_annotations_data['images'])

for img_no in range(end_no):
    
    annIds = val_coco.getAnnIds(imgIds=val_annotations_data['images'][img_no]['id'])
    anns = val_coco.loadAnns(annIds)

    # load and render the image
    plt.imshow(plt.imread(VAL_IMAGE_DIRECTIORY+val_annotations_data['images'][img_no]['file_name']))
    plt.axis('off')
    # Render annotations on top of the image
    val_coco.showAnns(anns)

    mask = val_coco.annToMask(anns[0])
    for i in range(len(anns)):
        mask += val_coco.annToMask(anns[i])
        plt.imshow(mask)
        plt.axis("off")
        plt.savefig(fname='D:/2022_imgdl/data/validation/masks/' + str(val_annotations_data['images'][img_no]['id']) + '.png', bbox_inches='tight', pad_inches=0)
    
    # plot 초기화
    plt.clf()

<Figure size 432x288 with 0 Axes>

In [31]:
# change file name
import os

path = "D:/2022_imgdl/data/validation/masks"
file_list = os.listdir(path)

for name in file_list:
    newname = name.replace(".png", "")
    newname = newname.zfill(6)

    newname = newname + ".png"
    src = os.path.join(path, name)
    dst = os.path.join(path, newname)
    os.rename(src, dst)

In [1]:
train_dir_path = "D:/2022_imgdl/data/training/images"
train_ids = os.listdir(train_dir_path)
val_dir_path = "D:/2022_imgdl/data/validation/images"
val_ids = os.listdir(val_dir_path)

In [2]:
# hyperparameter
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3
width_out = 128
height_out = 128

In [19]:
import numpy as np

x_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
y_train = np.zeros((len(train_ids), height_out, width_out, 1), dtype=bool)
x_val = np.zeros((len(val_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
y_val = np.zeros((len(val_ids), height_out, width_out, 1), dtype=bool)

In [64]:
import cv2

for i, id_ in enumerate(train_ids):
    img = cv2.imread('D:/2022_imgdl/data/training/images/'+id_)
    img = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH))
    x_train[i] = img

    mask = np.zeros((height_out,width_out,1), dtype=bool)
    mask_path = "D:/2022_imgdl/data/training/masks/"
    mask_ = cv2.imread(mask_path+id_[:6]+".png", 0)
    mask_ = cv2.resize(mask_, (height_out, width_out))
    mask_ = np.expand_dims(mask_, axis=-1)
    mask_ = np.where(mask_ >= 115, True, False)
    mask = np.maximum(mask,mask_)

    y_train[i] = mask

In [59]:
for i, id_ in enumerate(val_ids):
    img = cv2.imread('D:/2022_imgdl/data/validation/images/'+id_)
    img = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH))
    x_val[i] = img

    mask = np.zeros((height_out,width_out,1), dtype=bool)
    mask_path = "D:/2022_imgdl/data/validation/masks/"
    mask_ = cv2.imread(mask_path+id_[:6]+".png", 0)
    mask_ = cv2.resize(mask_, (height_out, width_out))
    mask_ = np.expand_dims(mask_, axis=-1)
    mask_ = np.where(mask_ >= 115, True, False)
    mask = np.maximum(mask,mask_)

    y_val[i] = mask

In [65]:
np.save("x_train.npy", x_train)
np.save("y_train.npy", y_train)
np.save("x_val.npy", x_val)
np.save("y_val.npy", y_val)